In [1]:
!pip install --upgrade scikit-learn==1.3.2
!pip install pydub
!pip install SpeechRecognition
!pip install googletrans==4.0.0-rc1

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
import joblib

nltk.download('omw')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

from IPython.display import display
import numpy as np
import pandas as pd
import json

from googletrans import Translator
from gensim.models import KeyedVectors

import speech_recognition as sr

from pydub import AudioSegment


# Cargar el modelo pre-entrenado de Word2Vec
model = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)

formato = {
    'rojo': "\033[1;91m{}\033[0m",
    'azul': "\033[1;94m{}\033[0m",
    'amarillo': "\033[1;33m{}\033[0m"
}

def traducir(palabra):
    translator = Translator()
    translation = translator.translate(palabra, src='es', dest='en')
    return translation.text

def encontrar_relacion(palabra1, palabra2):
    if palabra1 == palabra2:
        return 1
    # Comprobar si las palabras están en el vocabulario del modelo
    if palabra1 not in model.index_to_key or palabra2 not in model.index_to_key:
        relaciones = []
        for syn1 in wn.synsets(palabra1):
            for syn2 in wn.synsets(palabra2):
                similitud = syn1.path_similarity(syn2)
                if similitud is not None:
                    relaciones.append((syn1, syn2, similitud))
        if relaciones:
            df = pd.DataFrame(relaciones, columns=['syn1', 'syn2', 'similitud'])
            return df['similitud'].max()
        else:
            return 0

    similitud = model.similarity(palabra1, palabra2)

    return similitud

def procesar_entrada(texto):
    tokens = word_tokenize(texto)
    tokens = [word for word in tokens if word.isalnum()]
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word.lower() not in stop_words]
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word.lower()) for word in tokens]
    return tokens

def transcribe_audio(ruta_audio):

    file_to_wav(ruta_audio, "audio.wav")
    # Crear un objeto reconocedor de voz
    reconocedor = sr.Recognizer()

    # Leer el archivo de audio
    with sr.AudioFile("audio.wav") as audio:
        # Escuchar el audio usando el reconocedor
        audio = reconocedor.record(audio)
        try:
            # Transcribir el audio utilizando el modelo en español
            texto_transcrito = reconocedor.recognize_google(audio, language='es-ES')
            return texto_transcrito
        except sr.UnknownValueError:
            print("No se pudo entender el audio")
            return ""
        except sr.RequestError as e:
            print("Error al solicitar la transcripción; {0}".format(e))
            return ""

def file_to_wav(file, wav_file="audio.wav"):
    if file.endswith(".mp3"):
        audio = AudioSegment.from_mp3(file)
    elif file.endswith(".mp4"):
        audio = AudioSegment.from_file(file, format="mp4")
    audio.export(wav_file, format="wav")

[nltk_data] Downloading package omw to
[nltk_data]     C:\Users\rauls\AppData\Roaming\nltk_data...
[nltk_data]   Package omw is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\rauls\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rauls\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\rauls\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [12]:
palabra1, palabra2 = "car","bus"
relaciones = []
for syn1 in wn.synsets(palabra1):
    for syn2 in wn.synsets(palabra2):
        similitud = syn1.path_similarity(syn2)
        if similitud is not None:
            relaciones.append((syn1, syn2, similitud))
df = pd.DataFrame(relaciones, columns=['syn1', 'syn2', 'similitud'])
print(sorted(list(df['similitud']),reverse=True))
print(df['similitud'].max())

[0.5, 0.16666666666666666, 0.16666666666666666, 0.14285714285714285, 0.125, 0.1111111111111111, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.07692307692307693, 0.07692307692307693, 0.07692307692307693, 0.07692307692307693, 0.07692307692307693, 0.07142857142857142, 0.07142857142857142, 0.07142857142857142, 0.07142857142857142, 0.07142857142857142, 0.07142857142857142, 0.07142857142857142, 0.06666666666666667, 0.06666666666666667, 0.06666666666666667, 0.06666666666666667, 0.06666666666666667, 0.0625, 0.05555555555555555, 0.05263157894736842, 0.05263157894736842, 0.05263157894736842, 0.05]
0.5


In [2]:
# dicc.json to dicc
with open('dicc.json', 'r') as file:
    dicc = json.load(file)

In [3]:
# Read PrimerosAuxilios.txt
file = open("dolencias medicas copy.txt", "r")
primeros_auxilios = file.read().split("\n")
file.close()

textos = {}

target = ""
for x in primeros_auxilios:
  if '"' not in x:
    elemento = x.split(".")[-1].strip()
    if elemento!='':
      if elemento not in textos:
        textos[elemento]=[]
      target = elemento
  else:
      textos[elemento].append(x.split('"')[1])

In [6]:
def scan(objetive,frase,modelo):
    global dicc, formato, X, y, df_total
    tokens = list(set(procesar_entrada(frase)))

    claves,result = [x for x in dicc],[]
    columns_df = ["key","vector","model","comb","real"]

    for key in claves:
        dataset = procesar_entrada(key)+dicc[key]

        lista = [max([encontrar_relacion(x, y) for y in tokens]) for x in dataset]
        mean = np.mean(lista)
        n = (np.mean(lista)+np.max(lista))*np.mean(lista)
        lista += [mean for _ in range(11-len(lista))]

        X.append(lista)
        y.append(objetive==key)

        prob = modelo.predict_proba([lista])[0][1]

        result.append([key,n,prob,prob+n,objetive==key])

    df = pd.DataFrame(result,columns=columns_df)
    df_total.append(df)
    multi = 1/sum(df['comb'])
    df['comb'] = df['comb'].apply(lambda x: x*multi)
    df.sort_values(by="comb",inplace=True,ascending=False)

    claves = list(df.head(5)['key'].values)
    if objetive==claves[0]:
      print(formato["azul"].format("EXITO"))
      display(df.head(1))
    elif objetive in claves:
      print(formato["amarillo"].format("WARNING"))
      print(frase)
      print(tokens)
      display(df.head(5))
    else:
      print(formato["rojo"].format("ERROR"))
      print(frase)
      print(tokens)
      display(df)


    return X,y,df_total


X,y,df_total=[],[],[]
rfm = joblib.load('modelo_random_forest.pkl')

for key in textos:
    for f in textos[key]:
      print(key)
      frase = traducir(f).lower()
      for x in [",","."]: frase = frase.replace(x,f"{x} ")

      X,y,df_total = scan(key,frase,rfm)
      print()

chemical burn
EXITO


,key,vector,model,comb,real
0,chemical burn,0.873047,0.74,0.377338,True



chemical burn
EXITO


,key,vector,model,comb,real
0,chemical burn,0.238056,0.65,0.330044,True



chemical burn
EXITO


...

,key,vector,model,comb,real
3,food poisoning,0.753689,0.58,0.275278,False
11,hypoglycemia,0.825932,0.43,0.259228,True
8,choking,0.301272,0.09,0.080760,False
6,hemorrhage,0.142865,0.22,0.074896,False
9,cardiopulmonary,0.230830,0.11,0.070348,False



hypoglycemia
EXITO


,key,vector,model,comb,real
11,hypoglycemia,0.453411,0.03,0.244763,True



hypoglycemia
EXITO


,key,vector,model,comb,real
11,hypoglycemia,0.572863,0.58,0.434208,True


In [13]:
df = pd.DataFrame(X)
for i in range(len(df)):
    df.iloc[i] = df.iloc[i].fillna(df.iloc[i].mean())
df['y'] = y
df['y'] = df['y'].apply(lambda x: {True:1,False:0}[x])

X = df.drop('y', axis=1)
y = df['y'].values

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear un clasificador de Random Forest
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Entrenar el clasificador
rf_classifier.fit(X_train, y_train)

# Predecir las etiquetas para el conjunto de prueba
predictions = rf_classifier.predict(X_test)
probabilidades = rf_classifier.predict_proba(X_test)

# Supongamos que 'rf_classifier' es tu modelo entrenado

# Guardar el modelo en un archivo
joblib.dump(rf_classifier, 'modelo_random_forest2.pkl')

['modelo_random_forest2.pkl']

In [14]:
pd.set_option('display.max_rows', 300)

resultados = pd.DataFrame({'y_test': y_test, 'y_pred': predictions})
for i in range(probabilidades.shape[1]):
    resultados[f'prob_clase_{"False" if i==0 else "True"}'] = probabilidades[:, i]
display(resultados)

# Matriz de confusión
conf_matrix = confusion_matrix(y_test, predictions)
conf_matrix_df = pd.DataFrame(conf_matrix, index=rf_classifier.classes_, columns=rf_classifier.classes_)
display(conf_matrix_df)

reporte_clasificacion = classification_report(y_test, predictions, output_dict=True)
reporte_clasificacion_df = pd.DataFrame(reporte_clasificacion).transpose()
display(reporte_clasificacion_df)

display(resultados[(resultados['prob_clase_False']<0.8)|(resultados['y_test']!=resultados['y_pred'])])

,y_test,y_pred,prob_clase_False,prob_clase_True
0,0,0,0.99,0.01
1,0,0,1.00,0.00
2,0,0,1.00,0.00
3,0,0,0.89,0.11
4,0,0,0.99,0.01
...,...,...,...,...
370,0,0,1.00,0.00
371,1,1,0.31,0.69
372,0,0,1.00,0.00
373,0,0,0.99,0.01


,0,1
0,339,4
1,12,20


,precision,recall,f1-score,support
0,0.965812,0.988338,0.976945,343.000000
1,0.833333,0.625000,0.714286,32.000000
accuracy,0.957333,0.957333,0.957333,0.957333
macro avg,0.899573,0.806669,0.845615,375.000000
weighted avg,0.954507,0.957333,0.954532,375.000000


,y_test,y_pred,prob_clase_False,prob_clase_True
20,0,1,0.36,0.64
23,1,1,0.08,0.92
40,1,1,0.14,0.86
46,1,1,0.34,0.66
52,0,0,0.62,0.38
73,1,0,0.78,0.22
82,1,1,0.16,0.84
84,0,1,0.22,0.78
90,0,0,0.59,0.41
91,1,1,0.49,0.51
